# Get started

## Basic example: prompt + model + output parser

In [44]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

In [53]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-****************************************************"

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

### 1. Prompt

In [57]:
prompt_value = prompt.invoke({"topic": "ice cream"})


In [58]:
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [59]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [60]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

### 2. Model

In [61]:
message = model.invoke(prompt_value)

In [62]:
message

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [63]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nWhy did the ice cream go to therapy? Because it was feeling a little melon-coly.'

### 3. Output parser

In [64]:
output_parser.invoke(message)

"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

### 4. Entire Pipeline

In [65]:
input = {"topic": "ice cream"}

In [66]:
(prompt | model).invoke(input)

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're on the dance floor!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

## RAG Search Example

In [67]:
!pip --q install langchain docarray tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 4.4 MB/s eta 0:00:00


In [68]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser


/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [69]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'